# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.16 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [2]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]


### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -6.65278625 -1.5        -0.          0.
  6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
for i in range(1):                                         # play game for 5 episode
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.max(scores)))

Total score (averaged over agents) this episode: 0.0


When finished, you can close the environment.

In [6]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [7]:
%load_ext autoreload
%autoreload 2

In [9]:
from configparser import ConfigParser 
configur = ConfigParser() 
configur.read('config.ini')

Gamma = float(configur.get('agent','gamma'))

num_episodes = int(configur.get('train','number_of_episodes'))
random_seed = int(configur.get('train','random_seed'))
state_size = int(configur.get('env','state_size'))
action_size = int(configur.get('env','action_size'))
num_agents = int(configur.get('env','num_agents'))



In [10]:
from collections import namedtuple, deque

def trainAgent(magent,env,ne,brain_name):
    scores=[]
    window=deque(maxlen=100)
    ts=0
    noise=1
    decay=0.998
    for i_ep in range(1,ne+1):
        env_info = env.reset(train_mode=True)[brain_name]  
        states = env_info.vector_observations 
        score = np.zeros(num_agents) 
        [agent.reset() for agent in magent.agents]
        

        while True:
            ts+=1
            actions=magent.get_action(states,noise) 
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations        # get next state (for each agent)
            
            rewards = env_info.rewards                        # get reward (for each agent)
            dones = env_info.local_done                       # see if episode finished
            score += rewards                                 # update the score (for each agent)
            
            magent.step(states,actions,rewards,next_states,dones,ts)
            
            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
        noise= noise*decay
        scores.append(np.max(score))
        window.append(np.max(score))
        
        if i_ep %10 ==0 :
            print("Episode {}/{} Avg scores:{} last score :{}".format(i_ep,ne,np.mean(window),score))
        
        if np.mean(window) >= 0.5:
            print ("solved in {} episodes".format(i_ep))
            print ("Saving Model...")
            for j,agent in enumerate(magent.agents):
                torch.save(agent.actor_local.state_dict(), 'checkpoint_actor'+j+'.pth')
                torch.save(agent.critic.state_dict(), 'checkpoint_critic'+j+'.pth')
            break
        
    return scores

In [11]:
from MADDPG import MADDPG
magent= MADDPG(state_size,action_size,num_agents,Gamma,random_seed=10) 

In [12]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [ ]:
scores=trainAgent(magent,env,num_episodes,brain_name)
scores_copy=scores

Episode 10/1500 Avg scores:0.010000000149011612 last score :[ 0.   -0.01]
Episode 20/1500 Avg scores:0.014500000234693288 last score :[ 0.   -0.01]
Episode 30/1500 Avg scores:0.01300000020613273 last score :[ 0.1  -0.01]
Episode 40/1500 Avg scores:0.012250000191852451 last score :[-0.01  0.  ]
Episode 50/1500 Avg scores:0.015400000251829624 last score :[ 0.    0.09]
Episode 60/1500 Avg scores:0.014500000234693288 last score :[-0.01  0.  ]
Episode 70/1500 Avg scores:0.016428571726594652 last score :[-0.01  0.  ]
Episode 80/1500 Avg scores:0.014375000260770321 last score :[ 0.   -0.01]
Episode 90/1500 Avg scores:0.012777778009573618 last score :[ 0.   -0.01]
Episode 100/1500 Avg scores:0.013400000240653753 last score :[ 0.   -0.01]
Episode 110/1500 Avg scores:0.012400000225752592 last score :[ 0.   -0.01]
Episode 120/1500 Avg scores:0.012400000225752592 last score :[ 0.1  -0.01]
Episode 130/1500 Avg scores:0.013400000240653753 last score :[ 0.   -0.01]
Episode 140/1500 Avg scores:0.01340

In [ ]:
scores_copy=scores

In [ ]:
#graph of score over episodes
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

target_score=0.5
rolling_mean = pd.Series(scores_copy).rolling(window=10).mean()

plt.plot(range(len(scores_copy)),scores_copy,label="avg scores")
plt.plot(rolling_mean,label="rolling mean")
plt.plot(range(len(scores_copy)),(len(scores_copy)*[target_score]), linestyle='--',label="target", color='r')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend()
plt.grid("on")
plt.show()

In [ ]:
#to play
import torch
magent.agents[0].actor_local.load_state_dict(torch.load('checkpoint_actor-0.pth'))
magent.agents[0].critic_local.load_state_dict(torch.load('checkpoint_critic-0.pth'))
magent.agents[1].actor_local.load_state_dict(torch.load('checkpoint_actor-1.pth'))
magent.agents[1].critic_local.load_state_dict(torch.load('checkpoint_critic-1.pth'))


env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions=agent.get_action(states)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.max(scores)))

In [ ]:
env.close()